In [10]:
import os
os.environ["PATH"] += os.pathsep + r"/Users/mmymacymac/Developer/Tools/ffmpeg-2025-08-18-git-0226b6fb2c-full_build/bin"

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True,
    chunk_length_s=10,
    stride_length_s=2,
)

sample = "../audio/calling01.mp3"

result = pipe(sample)

print(result)

Device set to use cpu
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True). To use Whisper for long-form transcription, use rather the model's `generate` method directly as the model relies on it's own chunking mechanism (cf. Whisper original paper, section 3.8. Long-form Transcription).


In [ ]:
start_end_text = []
for chunk in result["chunks"]:
    start = chunk["timestamp"][0]
    end = chunk["timestamp"][1]
    text = chunk["text"]
    start_end_text.append([start, end, text])

import pandas as pd
df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
df.to_csv("calling01.csv", index=False, sep="|")
display(df)

,start,end,text
0,0.00,11.00,Музыка.
1,11.00,20.00,"상담사와 연결되었습니다. 어? 여보세요 안녕하세요 이름 말씀드리면 될까요? 네, ..."
2,20.00,22.00,"네, 이름 말씀드리면 될까요?"
3,22.00,24.00,"네, 생년월일 말씀해주세요."
4,24.00,26.00,"네, 11월 6일이요."
...,...,...,...
160,497.00,499.00,"응, 응. 그래."
161,499.00,502.00,"아, 네. 이도면 된 것 같아요"
162,504.00,506.02,감사합니다 아 그래 그래 감사합니다.
163,507.60,508.02,어 그래 그래 어.
